# Notebook 2: Conducting and Evaluating Regression Analysis

In [22]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import tqdm
import glob
import pandas as pd
import sklearn
from src import utils

np.set_printoptions(suppress=True)
from sklearn.linear_model import LinearRegression

np.random.seed(2)

Import datasets that were preprocessed in Notebook 1

In [23]:
wb_data = pd.read_csv("data/wb_data.csv")
wb_data_short = pd.read_csv("data/wb_data_short.csv")
whr_data = pd.read_csv("data/whr_data.csv")

wb_data.index = wb_data["Country Name"]
wb_data.drop(columns=["Country Name"], inplace=True)
wb_data_short.index = wb_data_short["Country Name"]
wb_data_short.drop(columns=["Country Name"], inplace=True)


whr_data.index = whr_data["Country name"]
whr_data.drop(columns=["Country name"], inplace=True)
#whr_data.head(20)

# sort by index
wb_data.sort_index(inplace=True)
wb_data_short.sort_index(inplace=True)
whr_data.sort_index(inplace=True)

In [24]:
# test: are the datasets equal?
print(sorted(list(wb_data.index))==sorted(list(whr_data.index)))

True


Standardize world bank data: $\frac{x-\mu}{\sigma}$

In [25]:
from sklearn.preprocessing import StandardScaler

wb_data_st = wb_data.copy(deep=True)
wb_data_short_st = wb_data_short.copy(deep=True)

wb_data_st[:] = StandardScaler().fit_transform(wb_data)
wb_data_short_st[:] = StandardScaler().fit_transform(wb_data_short)

In [27]:
# test: are the datasets equal
print(sorted(list(wb_data.index))==sorted(list(whr_data.index)))

True


In [28]:
# drop everything but life satisfaction ladder score from whr data
whr_scores = whr_data["Ladder score"]

## Pearson correlation coefficients
Aim: Get the correlation coefficient of each indicator with the whr_data

In [29]:
import scipy.stats

indicator_corr = wb_data_short.corr(method="pearson")
indicator_corr
indicator_corr[indicator_corr>0.8]

threshold = 0.85
#utils.print_corr(indicator_corr, threshold)
#TODO function that creates pretty colored corr matrix

"\nfor name, values in indicator_corr.iteritems():\n    print()\n    print('\nTarget indicator: ', name)\n    print('Correlated Indicators:')\n    for i in range(0, indicator_corr.shape[1]):\n        if threshold < values[i] < 1:\n            name = indicator_corr.columns[i]\n            print('{name}: {value}'.format(name=name, value=values[i]))\n"

Finding the VIF value for each indicator.

In [30]:
wb_vif = utils.sklearn_vif(wb_data.columns, wb_data)

/home/christian/Desktop/Data Literacy/Data-Literacy-project/src/utils.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
/home/christian/Desktop/Data Literacy/Data-Literacy-project/src/utils.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
/home/christian/Desktop/Data Literacy/Data-Literacy-project/src/utils.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
/home/christian/Desktop/Data Literacy/Data-Literacy-project/src/utils.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
/home/christian/Desktop/Data Literacy/Data-Literacy-project/src/utils.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
/home/christian/Desktop/Data Literacy/Data-Literacy-project/src/utils.py:25: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1/(1 - r_squared)
/home/christian/

Remove indicators that are too strongly correlated 

In [31]:
cc_indicators = list(wb_vif["VIF"].sort_values()[15:156].index)

wb_data_cc = wb_data.drop(columns=cc_indicators)
wb_data_cc.shape

(150, 15)

In [32]:
#TODO test whether 2-step reduction makes a differnece!

In [33]:
utils.sklearn_vif(wb_data_cc.columns, wb_data_cc)

,VIF,Tolerance
"Access to electricity, urban (% of urban population)",1.7,0.5886
Adjusted savings: mineral depletion (current US$),1.2,0.8191
"Incidence of tuberculosis (per 100,000 people)",1.7,0.6015
"Merchandise exports by the reporting economy, residual (% of total merchandise exports)",1.2,0.8213
Merchandise exports to economies in the Arab World (% of total merchandise exports),1.2,0.8662
Merchandise exports to high-income economies (% of total merchandise exports),1.6,0.6288
Merchandise exports to low- and middle-income economies in South Asia (% of total merchandise exports),1.4,0.7248
"Merchandise imports by the reporting economy, residual (% of total merchandise imports)",1.1,0.9125
Merchandise imports from low- and middle-income economies outside region (% of total merchandise imports),1.4,0.6996
Population density (people per sq. km of land area),1.6,0.6393


In [34]:
wb_data_cor = wb_data.corr()
wb_inv_corr = pd.DataFrame(np.linalg.inv(wb_data_cor.values), index = wb_data_cor.index, columns=wb_data_cor.columns)
#wb_inv_corr.diagonals
#print_correlations(indicator_corr)

## Split data into train and test set

In [36]:
test_size = 30
train, test, train_gt, test_gt = utils.split_data(wb_data_short_st, whr_scores, test_size)

In [37]:
print(train.shape, test.shape, train_gt.shape, test_gt.shape)
print(list(test.index)==list(test_gt.index))

(120, 120) (30, 120) (120,) (30,)
True


## Linear regression

Let's see how linear regression performs on wb_data and wb_data_short (redundant indicators removed).

In [38]:
# lets remove sex ratio lol
wb_data_cc = wb_data_cc.drop("Sex ratio at birth (male births per female births)", axis=1)

normalization worsens error from 0.7 to 1.3

In [39]:
wb_data_cc_norm = wb_data_cc.copy(deep=True)
wb_data_cc_norm[:] = sklearn.preprocessing.normalize(wb_data_cc)


In [40]:
loss_list, coef_list = utils.n_fold_ceval(1500, wb_data_cc, whr_scores, 30)
loss_arr = np.array(loss_list)
mean_loss = loss_arr.mean()

print(mean_loss)

0.7152410658941222


In [41]:
avg_coefs = np.around(np.mean(coef_list, axis=0), 2)

wb_data_cc.columns
#avg_coefs

Index(['Access to electricity, urban (% of urban population)',
       'Adjusted savings: mineral depletion (current US$)',
       'Incidence of tuberculosis (per 100,000 people)',
       'Merchandise exports by the reporting economy, residual (% of total merchandise exports)',
       'Merchandise exports to economies in the Arab World (% of total merchandise exports)',
       'Merchandise exports to high-income economies (% of total merchandise exports)',
       'Merchandise exports to low- and middle-income economies in South Asia (% of total merchandise exports)',
       'Merchandise imports by the reporting economy, residual (% of total merchandise imports)',
       'Merchandise imports from low- and middle-income economies outside region (% of total merchandise imports)',
       'Population density (people per sq. km of land area)',
       'Secondary education, duration (years)',
       'Secure Internet servers (per 1 million people)',
       'Statistical performance indicators (SP

In [42]:
avg_coefs[-12]
#avg_coefs

-0.0

For high n, the mse-loss is around 5. We print the first 50 entries of the loss array to check for outliers. It turns out that the variance is quite large and the loss is roughly in a range of [1, 25]